# **1. Environment Setup & Dependencies Installation**


In [ ]:
# Environment detection - simple and clear
import os
IN_COLAB = 'COLAB_GPU' in os.environ

if IN_COLAB:
    print("Installing dependencies for Colab environment...")
    !pip install -q unsloth "xformers<0.0.30" trl peft bitsandbytes
else:
    print("Local environment detected")
    !pip install -q unsloth

# Common libraries
!pip install -q datasets transformers accelerate


**Environment Setup**

This section automatically detects whether the code is running in a Google Colab environment.  
- If running in **Colab**, it installs all required libraries for fine-tuning (`unsloth`, `xformers`, `trl`, `peft`, `bitsandbytes`).  

**Installed libraries overview (Colab):**
- **unsloth:** Optimized fine-tuning framework for large language models (fast, memory-efficient, and 4-bit ready).  
- **xformers:** Meta’s optimized attention library that speeds up Transformer operations on GPUs.  
- **trl:** Hugging Face’s library for supervised and reinforcement learning-based fine-tuning.  
- **peft:** Parameter-Efficient Fine-Tuning toolkit (LoRA, prefix-tuning, etc.).  
- **bitsandbytes:** Enables 8-bit and 4-bit quantization for reduced GPU memory usage during training.  

- If running **locally**, it only installs the core `unsloth` package.  

**Common libraries installed in both cases:**
- **datasets:** Easy access to datasets from Hugging Face for training and evaluation.  
- **transformers:** Provides tokenizers and pretrained model classes for loading and training LLMs.  
- **accelerate:** Handles device placement, mixed precision, and distributed training efficiently.



# **2. Model Loading & Configuration**


In [ ]:
from unsloth import FastLanguageModel
import torch

# Model configuration
max_seq_length = 1024  # Reduced for T4 memory efficiency
dtype = None  # Auto detection: Float16 for T4, Bfloat16 for Ampere+
load_in_4bit = True  # Essential for T4 GPU memory

# Load model and tokenizer
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Llama-3.2-3B-Instruct-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...",  # Use if accessing gated models
)

**Model Loading & Configuration**

In this section, we load the **Llama-3.2-3B-Instruct** model using the `Unsloth` library, which provides optimized fine-tuning and inference performance for low-memory environments (like Google Colab T4 GPUs).

**What this code does:**
- **Imports:** Loads `FastLanguageModel` from `unsloth` and `torch` for tensor operations.
- **Model parameters:**
  - `max_seq_length = 1024` → Limits the model’s context window to 1024 tokens for better memory efficiency on T4 GPUs.  
    (It can be increased to 2048, but that would roughly double the memory usage and may cause OOM errors.)
  - `dtype = None` → Lets Unsloth automatically choose the most suitable precision (Float16 for T4, BFloat16 for newer GPUs).
  - `load_in_4bit = True` → Loads the model in 4-bit quantized format, significantly reducing GPU memory usage.

**Why this model?**

The **Llama-3.2-3B-Instruct** model offers a strong balance between:
- Quality (instruction-tuned for conversation and reasoning)
- Speed (smaller than 7B/13B models)
- Efficiency (fits within Colab’s free-tier GPU memory limits)

Overall, this setup ensures that fine-tuning can run smoothly and efficiently on limited GPU resources while still maintaining high model quality.



# **3. LoRA Configuration (Parameter-Efficient Fine-Tuning)**


In [ ]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 64,  # LoRA rank - balanced for T4
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = 128,  # 2x rank for better learning
    lora_dropout = 0,  # Optimized: 0 for better performance
    bias = "none",  # Optimized: no bias training
    use_gradient_checkpointing = "unsloth",  # Memory efficient
    random_state = 3407,  # Reproducibility seed
    use_rslora = True,  # Rank-Stabilized LoRA for stability
    loftq_config = None,
)

**LoRA Configuration (Parameter-Efficient Fine-Tuning)**

In this section, the pre-loaded Llama model is converted into a **LoRA-enabled fine-tuning model** using the `Unsloth` PEFT integration.  
Instead of training *all* model weights, LoRA updates only a few low-rank adapter layers — making training much faster and memory-efficient.

**What this code does:**
- Uses `FastLanguageModel.get_peft_model()` to wrap the model with LoRA layers.
- Targets specific transformer layers (`q_proj`, `k_proj`, `v_proj`, `o_proj`, etc.) that are most influential in learning context and attention patterns.

**Parameter explanations:**
- **`r = 64`** → LoRA rank. Controls how much capacity the adapters have to learn new information.  
  A value of 64 provides a balanced trade-off between learning power and memory efficiency on T4 GPUs.
- **`lora_alpha = 128`** → Scaling factor, typically set to 2× the rank for stable training.
- **`lora_dropout = 0`** → No dropout applied; this yields better performance for smaller datasets.
- **`use_gradient_checkpointing = "unsloth"`** → Saves GPU memory by recomputing intermediate values on demand.
- **`use_rslora = True`** → Enables *Rank-Stabilized LoRA* for more stable convergence.
- **`bias = "none"`** → Bias parameters are not trained to keep optimization simple and efficient.

**Why this setup?**

This configuration is specifically tuned for **Google Colab T4 GPUs**:
- 4-bit quantization keeps the model lightweight.
- LoRA reduces the number of trainable parameters by ~99%.
- The chosen rank and modules ensure a strong learning signal without exceeding memory limits.

Overall, this makes fine-tuning feasible on limited hardware while still achieving high-quality results.


# **4. Dataset Loading & Preprocessing**


**Dataset Loading & Preprocessing**

In this section, we prepare the dataset for fine-tuning by:

- Loading the ```Python Code Instructions 18K Alpaca``` dataset from **Hugging Face**.

- Exploring its structure, which includes instruction, input, and output fields designed for instruction-based code generation tasks.

- Inspecting sample entries to understand how programming-related prompts and solutions are formatted.


In [ ]:
# Load medical dataset
from datasets import load_dataset

dataset = load_dataset("medalpaca/medical_meadow_medical_flashcards", split="train")

# Dataset info
print(f"Dataset size: {len(dataset)}")
print(f"Columns: {dataset.column_names}")
print("\n" + "="*50)

# Show first 3 examples
for i in range(3):
    print(f"\nExample {i+1}:")
    print(dataset[i])


## **4.1. Split Dataset**

In [ ]:
# Split dataset - take 10% for efficient training
dataset = dataset.train_test_split(train_size=0.1)["train"]

print(f"Training samples: {len(dataset)}")
print(f"\nFirst example after split:")
print(dataset[0])

## **4.2. Alpaca Prompt Format**

In [ ]:
# Medical Alpaca prompt
alpaca_prompt = """You are an expert doctor in the field of medicine.
Answer the following medical question professionally and accurately.

### Question:
{}

### Medical Answer:
{}"""

EOS_TOKEN = tokenizer.eos_token

def formatting_medical(examples):
    inputs = examples["input"]
    outputs = examples["output"]
    texts = []

    for inp, out in zip(inputs, outputs):
        text = alpaca_prompt.format(inp, out) + EOS_TOKEN
        texts.append(text)

    return {"text": texts}

In [ ]:
# 2️⃣ Define formatting function
dataset = dataset.map(formatting_medical, batched=True)

In [ ]:
dataset

In [ ]:
# Check first example
print(dataset[0]['text'][:500])

## **4.1. Mount Google Drive**

In [ ]:
# Mount Google Drive for saving model
from google.colab import drive
drive.mount('/content/drive')

**Model Saving Setup (Google Drive)**

Google Drive is mounted to store fine-tuned models and checkpoints.  
This ensures training outputs are not lost when the Colab runtime resets.


# **5. Training Configuration & Initialization**

In [ ]:
# Training configuration - MEDICAL MODEL
from unsloth import UnslothTrainer, UnslothTrainingArguments

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,

    args = UnslothTrainingArguments(
        # Optimize for speed and quality
        per_device_train_batch_size = 6,
        gradient_accumulation_steps = 2,

        # Full epoch training
        num_train_epochs = 1,
        warmup_ratio = 0.05,

        # Learning rates
        learning_rate = 5e-5,
        embedding_learning_rate = 1e-6,

        # Optimization
        optim = "adamw_8bit",
        weight_decay = 0.01,
        lr_scheduler_type = "cosine",
        seed = 3407,

        # Logging and saving
        logging_steps = 30,
        save_strategy = "steps",
        save_steps = 150,
        save_total_limit = 2,

        # Output
        output_dir = "/content/drive/MyDrive/unsloth_medical_assistant",
        report_to = "none",
    ),
)

## **5.1. Show Current Memory Stats**

In [ ]:
# Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

## **5.2. Start Training**

In [ ]:
# Start Training
trainer_stats = trainer.train()

## **5.3. Save Trained Model**

In [ ]:
model.save_pretrained("/content/drive/MyDrive/unsloth_medical_final")
tokenizer.save_pretrained("/content/drive/MyDrive/unsloth_medical_final")

# **6. Inference / Text Generation**


In [ ]:
FastLanguageModel.for_inference(model)

alpaca_prompt = """You are an expert doctor in the field of medicine.
Answer the following medical question professionally and accurately.

### Question:
{}

### Medical Answer:
"""

EOS_TOKEN = tokenizer.eos_token


# ============================================================
# ✅ 1. Single sanity test (optional)
# ============================================================
test_prompt = alpaca_prompt.format("What are the common symptoms of diabetes?")

inputs = tokenizer(test_prompt, return_tensors="pt").to("cuda")

outputs = model.generate(
    **inputs,
    max_new_tokens=200,
    temperature=0.5,
    top_p=0.9,
    repetition_penalty=1.2,
    do_sample=True,
    pad_token_id=tokenizer.eos_token_id,
)

print("\n🩺 Single test output:\n")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


# ============================================================
# ✅ 2. Multi-question test set (Alpaca format)
# ============================================================
test_questions = [
    "What are the common symptoms of diabetes?",
    "What is hypertension?",
    "What causes migraine headaches?",
    "Explain the function of insulin in the human body.",
    "What are the side effects of antibiotics?",
    "How does dehydration affect kidney function?",
    "What are the risk factors for heart disease?",
    "How does the immune system respond to viral infections?",
    "Explain the difference between Type 1 and Type 2 diabetes.",
    "What are the benefits of regular exercise for cardiovascular health?",
]

results = []

for i, question in enumerate(test_questions, start=1):
    prompt = alpaca_prompt.format(question)
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")

    outputs = model.generate(
        **inputs,
        max_new_tokens=200,
        temperature=0.5,
        top_p=0.9,
        repetition_penalty=1.2,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id,
    )

    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    results.append({"question": question, "answer": answer})
    print(f"\n[{i}] Question: {question}\nAnswer:\n{answer[:500]}\n{'-'*80}")
